<a href="https://colab.research.google.com/github/Chiranjeevi2001/coberx-gemma2b/blob/main/Gemma2bFinetuningCOBREX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install peft datasets transformers trl accelerate bitsandbytes -q
import torch
import json
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, PeftModelForCausalLM
from trl import SFTTrainer
from datasets import Dataset
from huggingface_hub import notebook_login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [5]:
save_path = "/content/dataset_cbl.json"
dataset_train_name = 'EN-train'
# dataset_val_name = 'EN-val'
# file_name_train_chatml = f"{dataset_train_name}_chatml.json"
# file_name_val_chatml = f"{dataset_val_name}_chatml.json"

with open(save_path, 'r', encoding='latin-1') as f:
  dataset_train = Dataset.from_dict(json.load(f))

# we will keep this as optional
# with open(save_path + file_name_val_chatml, 'r') as f:
#   dataset_val = Dataset.from_dict(json.load(f))

In [8]:
notebook_login()

In [9]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.float16,
)
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
tokenizer.padding_side = 'right'
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2b-it",
    device_map="auto",
    quantization_config=bnb_config
)

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [10]:
training_arguments = TrainingArguments(
    output_dir = "/content/output_adapters/",
    # evaluation_strategy="steps",
    logging_strategy="steps",
    lr_scheduler_type="constant",
    logging_steps=20,
    # eval_steps=20,
    save_steps=20,
    per_device_train_batch_size=2,
    # per_device_eval_batch_size=2,
    gradient_accumulation_steps=16,
    # eval_accumulation_steps=16,
    num_train_epochs=1,
    fp16=True,
    group_by_length = True,
    optim="paged_adamw_32bit",
    max_steps = 100
)
peft_config = LoraConfig(
    lora_alpha = 16,
    lora_dropout=0.1,
    r=16,
    task_type='CAUSAL_LM'
)
trainer = SFTTrainer(
    model,
    tokenizer=tokenizer,
    train_dataset=dataset_train,
    # eval_dataset=dataset_val,
    peft_config=peft_config,
    neftune_noise_alpha=5,
    max_seq_length=500,
    args = training_arguments
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:256: UserWarning: You passed a `neftune_noise_alpha` argument to the SFTTrainer, the value you passed will override the one in the `TrainingArguments`.
  warnings.warn(


Map:   0%|          | 0/393 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [11]:
trainer.train()

Step,Training Loss
20,2.885000
40,2.546000
60,2.147900
80,1.984500
100,1.810100


TrainOutput(global_step=100, training_loss=2.274683723449707, metrics={'train_runtime': 1508.1853, 'train_samples_per_second': 2.122, 'train_steps_per_second': 0.066, 'total_flos': 1.696283500713984e+16, 'train_loss': 2.274683723449707, 'epoch': 8.12})

In [12]:
trainer.model.save_pretrained("/content/output_adapters")

In [13]:
finetuned_model = PeftModelForCausalLM.from_pretrained(model=model, model_id="/content/output_adapters")

In [14]:
# Open the file in read mode
with open('codeSystemMessage.txt', 'r') as file:
    # Read the entire file content into a string
    text = file.read()

# Now 'text' contains the content of the file
print(text)


# you take a COBOL program and create Business Rules for it.
# you are GREAT at extracting the business rules from all business variables in the code.
# A business rules are defined as everyday business logics that get coded into the program.
# There is a clear distinction between a business rule and a trivial validation. A business rule MAY CHANGE over time, but a trivial validation does not.
# DO NOT INCLUDE ANYTHING OTHER THAN THE JSON in your response.
# You  must generate multiple business rules from the given program.
# mention the business rule id in your response.
# your answer MUST be a valid json format, with each property of a business rule clearly listed.
# any '\n' in the text fields MUST be '\\n' so that when reading it later on, we won't run into any issues

# example input and output with a simple COBOL file
# Input:
       IDENTIFICATION DIVISION.
       PROGRAM-ID. EXAMPLE.
       DATA DIVISION.
       WORKING-STORAGE SECTION.
       77  NUM         PICTURE 99.
      

In [15]:

messages = [
    {
        'role':'user',
        'content': text+"IDENTIFICATIONDIVISION.\nPROGRAM-ID.PROD2V1.\nENVIRONMENTDIVISION.\nCONFIGURATIONSECTION.\nSPECIAL-NAMES.\nINPUT-OUTPUTSECTION.\nFILE-CONTROL.\nSELECTRENTALASSIGNTO'RENTACAR-IN.txt'\nORGANISATIONISLINESEQUENTIAL.\nSELECTRENTAL-OUTASSIGNTO'RENTACAR-OUT.txt'.\nDATADIVISION.\nFILESECTION.\nFDRENTAL.\n01RENTAL-FILE.\n02CLIENT_NAMEPICA(20).\n02RENTAL-TYPE.\n03NAME_INITIALPICA(1).\n03CAR_TYPEPIC9(1).\n03KILOMETERSPIC9(5).\n03NUM_DAYSPIC9(3).\nFDRENTAL-OUT.\n01RENTAL-FILE-OUT.\n02CLIENT_NAME_OUTPICA(20).\n02FILLERPICX(1)VALUESPACE.\n02NAME_INITIAL_OUTPICA(1).\n02FILLERPICX(1)VALUESPACE.\n02CAR_BRANDPICA(10).\n02FILLERPICX(1)VALUESPACE.\n02KILOMETERS_OUTPICzzz99.\n02FILLERPICX(1)VALUESPACE.\n02NUM_DAYS_OUTPICzz9.\n02FILLERPICX(1)VALUESPACE.\n02PAYMENTPICzz99.99.\nWORKING-STORAGESECTION.\n77END-LOOPPICA(3)VALUESPACES.\n77KILOMETERS_PAYMENTPIC9(4)V99.\n77RENTAL_DAYS_TOTALPIC9(4).\nPROCEDUREDIVISION.\nMAIN-PROCEDURE.\nOPENINPUTRENTAL\nOUTPUTRENTAL-OUT.\nREADRENTAL\nATENDMOVE\"YYY\"TOEND-LOOP\nEND-READ.\nPERFORMCALCULATIONSUNTILEND-LOOP=\"YYY\".\nCLOSERENTAL\nRENTAL-OUT.\nSTOPRUN.\nCALCULATIONS.\nMOVECLIENT_NAMETOCLIENT_NAME_OUT.\nMOVENAME_INITIALTONAME_INITIAL_OUT.\nMOVEKILOMETERSTOKILOMETERS_OUT.\nMOVENUM_DAYSTONUM_DAYS_OUT.\nIFKILOMETERSISGREATERTHAN75OREQUALTO75\nCOMPUTEKILOMETERS=KILOMETERS-75.\nEVALUATECAR_TYPE\nWHEN1MOVE\"VOLKSWAGEN\"TOCAR_BRAND\nCOMPUTEKILOMETERS_PAYMENT=KILOMETERS*0.5\nCOMPUTERENTAL_DAYS_TOTAL=10*NUM_DAYS\nCOMPUTEPAYMENT=KILOMETERS_PAYMENT+RENTAL_DAYS_TOTAL\nWHEN2MOVE\"TOYOTA\"TOCAR_BRAND\nCOMPUTEKILOMETERS_PAYMENT=KILOMETERS*0.55\nCOMPUTERENTAL_DAYS_TOTAL=12.5*NUM_DAYS\nCOMPUTEPAYMENT=KILOMETERS_PAYMENT+RENTAL_DAYS_TOTAL\nWHEN3MOVE\"MERCEDES\"TOCAR_BRAND\nCOMPUTEKILOMETERS_PAYMENT=KILOMETERS*0.65\nCOMPUTERENTAL_DAYS_TOTAL=16*NUM_DAYS\nCOMPUTEPAYMENT=KILOMETERS_PAYMENT+RENTAL_DAYS_TOTAL\nEND-EVALUATE.\nWRITERENTAL-FILE-OUT\nAFTERSADVANCING1LINE.\nDISPLAYRENTAL-FILE-OUT.\nREADRENTAL\nATENDMOVE\"YYY\"TOEND-LOOP\nEND-READ.\nLEGACY.\nSTOPRUN.\nENDPROGRAMPROD2V1.\n"
    }
]

In [16]:
input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=True, return_tensors="pt").to("cuda")

# print(input_ids)
outputs_finetuned = finetuned_model.generate(input_ids=input_ids, max_new_tokens=1024, do_sample=False)
# outputs = model_2.generate(input_ids=input_ids, max_new_tokens=1024, do_sample=False)

print("finetuned: " + tokenizer.decode(outputs_finetuned[0]).split('<start_of_turn>model\n')[-1])
# print("normal   : " + tokenizer.decode(outputs[0]).split('<start_of_turn>model\n')[-1])

finetuned: {
  "id": "BR-001",
  "description": "A number is even if the remainder after dividing by 2 is 0, otherwise it is odd.",
  "condition": "NUM DIV 2",
  "output": {
    "even": "NUM is EVEN",
    "odd": "NUM is ODD"
  }
}
}<eos>
